In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- Step 0: Setup and Configuration ---

print("--- Starting Full Population Projection Model (2022-2030) ---")

# Define the start and end years for the projection
START_YEAR = 2022
END_YEAR = 2030

# Create the output directory if it doesn't exist to avoid errors.
output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the full paths to your input files.
base_pop_path = 'input/CSO_Base_Dataset_PEC19.20250604T160659.csv'
fertility_path = 'input/Fertility_Rate_VSA104.20250613T230625.csv'
migration_path = 'input/Migration_Rate_PEA03.20250617T200610.csv'
mortality_path = 'input/Mortality_Rate_VSA111.20250617T230624.csv'

# --- Data Cleaning Helper Function ---
def clean_age_column(age_series):
    """Cleans the age column to make it a numeric type."""
    age_series = age_series.str.replace('Under 1 year', '0', regex=False)
    age_series = age_series.str.replace(' years and over', '', regex=False)
    age_series = age_series.str.replace(' years', '', regex=False)
    age_series = age_series.str.replace(' year', '', regex=False)
    return pd.to_numeric(age_series)

# --- Load and Prepare All Base Data ---
print("Loading and preparing all base data...")
ages = np.arange(100) # Standard age range from 0 to 99

# Population
pop_df = pd.read_csv(base_pop_path)
pop_df['Age'] = clean_age_column(pop_df['Age'])
pop_df = pop_df.rename(columns={'VALUE': 'Population'})
male_pop_base = pop_df[pop_df['Sex'] == 'Male'].set_index('Age')['Population'].reindex(ages).fillna(0)
female_pop_base = pop_df[pop_df['Sex'] == 'Female'].set_index('Age')['Population'].reindex(ages).fillna(0)

# Fertility
fertility_df = pd.read_csv(fertility_path)
fertility_df['Age'] = clean_age_column(fertility_df['Age'])
fertility_rates = fertility_df.set_index('Age')['FertilityRatePer1000Women'].reindex(ages).fillna(0)

# Migration
migration_df = pd.read_csv(migration_path)
migration_df['Age'] = clean_age_column(migration_df['Age'])
male_mig_rates = migration_df[migration_df['Sex'] == 'Male'].set_index('Age')['NetMigrationRateInThousands'].reindex(ages).fillna(0)
female_mig_rates = migration_df[migration_df['Sex'] == 'Female'].set_index('Age')['NetMigrationRateInThousands'].reindex(ages).fillna(0)

# Mortality
mortality_df = pd.read_csv(mortality_path)
mortality_df = mortality_df.rename(columns={'AgeAtDeathIn2022': 'Age'})
mortality_df['Age'] = clean_age_column(mortality_df['Age'])
mortality_df['Deaths'] = pd.to_numeric(mortality_df['Deaths'], errors='coerce').fillna(0)
male_deaths = mortality_df[mortality_df['Sex'] == 'Male'].set_index('Age')['Deaths'].reindex(ages).fillna(0)
female_deaths = mortality_df[mortality_df['Sex'] == 'Female'].set_index('Age')['Deaths'].reindex(ages).fillna(0)
print("Data loading complete.")

# --- Task 1: Calculate Constant Demographic Rates ---
print("\n--- Task 1: Calculating Constant Demographic Rates ---")

# Migration numbers (constant based on base year population)
male_net_migrants = male_pop_base * (male_mig_rates / 1000)
female_net_migrants = female_pop_base * (female_mig_rates / 1000)

# Survival rates (constant based on base year population and deaths)
male_mortality_rate = (male_deaths / (male_pop_base + 1e-9)).clip(0, 1)
female_mortality_rate = (female_deaths / (female_pop_base + 1e-9)).clip(0, 1)
male_survival_rate = 1 - male_mortality_rate
female_survival_rate = 1 - female_mortality_rate
print("Constant migration and survival rates calculated.")


# --- Generic Projection Function ---
def run_projection(start_year, end_year, male_pop_base, female_pop_base,
                   fertility_rates, male_net_migrants, female_net_migrants,
                   male_survival_rate, female_survival_rate,
                   use_migration, use_mortality):
    """
    Runs a population projection based on the specified components.
    """
    male_proj_df = pd.DataFrame({start_year: male_pop_base})
    female_proj_df = pd.DataFrame({start_year: female_pop_base})

    # Use 100% survival if mortality is turned off
    male_survival = male_survival_rate if use_mortality else pd.Series(1.0, index=ages)
    female_survival = female_survival_rate if use_mortality else pd.Series(1.0, index=ages)

    for year in range(start_year + 1, end_year + 1):
        prev_year = year - 1
        male_pop_prev = male_proj_df[prev_year]
        female_pop_prev = female_proj_df[prev_year]

        male_pop_curr = pd.Series(0, index=ages)
        female_pop_curr = pd.Series(0, index=ages)

        # a) Apply Survival and Aging
        for age in range(len(ages) - 1):
            male_pop_curr[age + 1] = male_pop_prev[age] * male_survival[age]
            female_pop_curr[age + 1] = female_pop_prev[age] * female_survival[age]
        
        last_age = len(ages) - 1
        male_pop_curr[last_age] += male_pop_prev[last_age] * male_survival[last_age]
        female_pop_curr[last_age] += female_pop_prev[last_age] * female_survival[last_age]

        # b) Add Net Migration (if enabled)
        if use_migration:
            male_pop_curr += male_net_migrants
            female_pop_curr += female_net_migrants
        
        # c) Calculate and Add Newborns
        total_births = (female_pop_prev * (fertility_rates / 1000)).sum()
        male_newborns = total_births / 2
        female_newborns = total_births / 2
        
        male_pop_curr[0] = male_newborns * male_survival[0]
        female_pop_curr[0] = female_newborns * female_survival[0]

        # d) Final cleanup and store
        male_proj_df[year] = male_pop_curr.clip(0)
        female_proj_df[year] = female_pop_curr.clip(0)
        
    return male_proj_df, female_proj_df

# --- Task 2: Run and Save Each Scenario ---

# --- Scenario 1: Fertility Only ---
print("\n--- Running Scenario 1: Fertility Only ---")
male_proj_fert, female_proj_fert = run_projection(
    START_YEAR, END_YEAR, male_pop_base, female_pop_base, fertility_rates,
    male_net_migrants, female_net_migrants, male_survival_rate, female_survival_rate,
    use_migration=False, use_mortality=False
)
excel_path_fert = os.path.join(output_dir, "Projection_Fertility.xlsx")
with pd.ExcelWriter(excel_path_fert) as writer:
    female_proj_fert.to_excel(writer, sheet_name="Female")
    male_proj_fert.to_excel(writer, sheet_name="Male")
print(f"Saved Fertility-Only projection to {excel_path_fert}")


# --- Scenario 2: Fertility and Migration ---
print("\n--- Running Scenario 2: Fertility and Migration ---")
male_proj_fert_mig, female_proj_fert_mig = run_projection(
    START_YEAR, END_YEAR, male_pop_base, female_pop_base, fertility_rates,
    male_net_migrants, female_net_migrants, male_survival_rate, female_survival_rate,
    use_migration=True, use_mortality=False
)
excel_path_fert_mig = os.path.join(output_dir, "Projection_Fertility_Migration.xlsx")
with pd.ExcelWriter(excel_path_fert_mig) as writer:
    female_proj_fert_mig.to_excel(writer, sheet_name="Female")
    male_proj_fert_mig.to_excel(writer, sheet_name="Male")
print(f"Saved Fertility+Migration projection to {excel_path_fert_mig}")


# --- Scenario 3: Full Model (Fertility, Migration, Mortality) ---
print("\n--- Running Scenario 3: Full Model ---")
male_proj_full, female_proj_full = run_projection(
    START_YEAR, END_YEAR, male_pop_base, female_pop_base, fertility_rates,
    male_net_migrants, female_net_migrants, male_survival_rate, female_survival_rate,
    use_migration=True, use_mortality=True
)
excel_path_full = os.path.join(output_dir, "Projection_Full_Model.xlsx")
with pd.ExcelWriter(excel_path_full) as writer:
    female_proj_full.to_excel(writer, sheet_name="Female")
    male_proj_full.to_excel(writer, sheet_name="Male")
print(f"Saved Full Model projection to {excel_path_full}")


print("\n--- All Scenarios Completed Successfully! ---")


--- Starting Full Population Projection Model (2022-2030) ---
Loading and preparing all base data...
Data loading complete.

--- Task 1: Calculating Constant Demographic Rates ---
Constant migration and survival rates calculated.

--- Running Scenario 1: Fertility Only ---


C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:123: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27224.798200000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  male_pop_curr[0] = male_newborns * male_survival[0]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27224.798200000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  female_pop_curr[0] = female_newborns * female_survival[0]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27224.798200000005' has dtype incompatible with int64, please explicitly cast to 

Saved Fertility-Only projection to output\Projection_Fertility.xlsx

--- Running Scenario 2: Fertility and Migration ---


C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27224.798200000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  male_pop_curr[age + 1] = male_pop_prev[age] * male_survival[age]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27224.798200000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  female_pop_curr[age + 1] = female_pop_prev[age] * female_survival[age]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '26726.886333222' has dtype incompatible with int64, ple

Saved Fertility+Migration projection to output\Projection_Fertility_Migration.xlsx

--- Running Scenario 3: Full Model ---


C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '29438.000000000004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  male_pop_curr[age + 1] = male_pop_prev[age] * male_survival[age]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28058.000000000004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  female_pop_curr[age + 1] = female_pop_prev[age] * female_survival[age]
C:\Users\chinm\AppData\Local\Temp\ipykernel_6204\1021596697.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27026.131417291675' has dtype incompatible with int64, 

Saved Full Model projection to output\Projection_Full_Model.xlsx

--- All Scenarios Completed Successfully! ---


In [2]:
# --- Create and Save Animated Population Pyramid ---

print("\n--- Creating Animated Population Pyramid Visualization ---")

# Define the years to plot
plot_years = [2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

# Set up the figure and axis for the animation
fig, ax = plt.subplots(figsize=(10, 8))

def animate(year):
    """
    This function is called for each frame of the animation.
    It clears the previous plot and draws the new one for the given year.
    """
    # Get the data for the specific year
    male_data = male_proj_full[year]
    female_data = female_proj_full[year]
    
    # Clear the previous frame to prepare for the new one
    ax.clear()
    
    # Plot the horizontal bars
    # We plot male data as negative to put it on the left side
    ax.barh(ages, -male_data, color='royalblue', label='Male', height=1.0)
    ax.barh(ages, female_data, color='lightcoral', label='Female', height=1.0)
    
    # --- Formatting for each frame ---
    ax.set_title(f"Population Pyramid for {year}", fontsize=16, pad=20)
    ax.set_xlabel("Population", fontsize=12)
    ax.set_ylabel("Age", fontsize=12)
    
    # CORRECTED X-AXIS HANDLING:
    # 1. Define the tick locations manually to ensure 0 is in the center.
    max_pop = 45000
    tick_interval = 10000
    ax.set_xlim(-max_pop, max_pop)
    ticks = np.arange(-max_pop, max_pop + 1, tick_interval)
    ax.set_xticks(ticks)
    
    # 2. Create labels that are the absolute values of the tick locations.
    tick_labels = [f'{abs(int(tick)):,}' for tick in ticks]
    ax.set_xticklabels(tick_labels)
    
    # Set y-axis to show ages 0-99
    ax.set_ylim(-1, 100) 
    
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Add a vertical line at 0 for clarity
    ax.axvline(0, color='black', linewidth=0.8)

# Create the animation object
# The 'frames' argument tells the animation which years to cycle through
# The 'interval' is the time in milliseconds between frames
ani = animation.FuncAnimation(fig, animate, frames=plot_years, interval=1200, repeat=True)

# Save the animation as a GIF
# Note: You might need to install 'pillow': pip install pillow
pyramid_path = os.path.join(output_dir, "Population_Pyramid_Animation_.gif")
# Use the Pillow writer for reliable GIF creation
ani.save(pyramid_path, writer='pillow', fps=1)

# Add a final layout adjustment and show the plot for interactive environments
plt.tight_layout(pad=2.0)
plt.close(fig) # Close the figure to prevent it from displaying statically

print(f"Successfully saved animated population pyramid to {pyramid_path}")




--- Creating Animated Population Pyramid Visualization ---
Successfully saved animated population pyramid to output\Population_Pyramid_Animation_.gif


In [3]:
print("\n--- Calculating Prevalence")

# Step 1: Load the pre-processed prevalence rates
rates_path = 'input/PrevalenceRates.csv'
rates_df = pd.read_csv(rates_path)

# Step 2: Separate the rates into two distinct series, indexed by age
male_prevalence_rates = rates_df[rates_df['Sex'] == 'Male'].set_index('Age')['PrevalenceRate']
female_prevalence_rates = rates_df[rates_df['Sex'] == 'Female'].set_index('Age')['PrevalenceRate']

# Step 3: Calculate prevalent cases for each year
male_prevalent_cases = male_proj_full.mul(male_prevalence_rates, axis='index')
female_prevalent_cases = female_proj_full.mul(female_prevalence_rates, axis='index')

# Step 4: Create a summary and save the output
summary = {
    'Male Prevalent Cases': male_prevalent_cases.sum(),
    'Female Prevalent Cases': female_prevalent_cases.sum()
}
prevalence_summary_df = pd.DataFrame(summary)
prevalence_summary_df['Total Prevalent Cases'] = prevalence_summary_df.sum(axis=1)
prevalence_summary_df = prevalence_summary_df.round(0).astype(int)
prevalence_summary_df.index.name = 'Year'

print("\nProjected Prevalence :")
print(prevalence_summary_df)

# Save the detailed results to a multi-sheet Excel file
excel_path_final = os.path.join(output_dir, "Projected_Prevalence.xlsx")
with pd.ExcelWriter(excel_path_final) as writer:
    male_prevalent_cases.round(2).to_excel(writer, sheet_name="Male Cases")
    female_prevalent_cases.round(2).to_excel(writer, sheet_name="Female Cases")
    prevalence_summary_df.to_excel(writer, sheet_name="Summary")

print(f"\nSuccessfully saved prevalence projection to {excel_path_final}")




--- Calculating Prevalence

Projected Prevalence :
      Male Prevalent Cases  Female Prevalent Cases  Total Prevalent Cases
Year                                                                     
2022                 93414                   69360                 162774
2023                 95365                   70830                 166195
2024                 97295                   72280                 169575
2025                 99333                   73850                 173183
2026                101315                   75421                 176736
2027                103313                   77013                 180326
2028                105175                   78559                 183734
2029                106934                   80044                 186978
2030                108659                   81508                 190167

Successfully saved prevalence projection to output\Projected_Prevalence.xlsx


In [4]:
print("\n--- Calculating Incidence ---")

# Step 1: Load the pre-processed incidence rates
inc_rates_path = 'input/IncidenceRates.csv'
inc_rates_df = pd.read_csv(inc_rates_path)

# Step 2: Separate the rates into two distinct series, indexed by age
male_incidence_rates = inc_rates_df[inc_rates_df['Sex'] == 'Male'].set_index('Age')['IncidenceRate']
female_incidence_rates = inc_rates_df[inc_rates_df['Sex'] == 'Female'].set_index('Age')['IncidenceRate']

# Step 3: Calculate the population at risk for each year
male_pop_at_risk = male_proj_full - male_prevalent_cases
female_pop_at_risk = female_proj_full - female_prevalent_cases

# Step 4: Calculate the new (incident) cases for each year
male_incident_cases = male_pop_at_risk.mul(male_incidence_rates, axis='index')
female_incident_cases = female_pop_at_risk.mul(female_incidence_rates, axis='index')

# Step 5: Create a summary, excluding the base year 2022
inc_summary = {
    'Male Incident Cases': male_incident_cases.sum().loc[START_YEAR+1:],
    'Female Incident Cases': female_incident_cases.sum().loc[START_YEAR+1:]
}
incidence_summary_df = pd.DataFrame(inc_summary)
incidence_summary_df['Total Incident Cases'] = incidence_summary_df.sum(axis=1)
incidence_summary_df = incidence_summary_df.round(0).astype(int)
incidence_summary_df.index.name = 'Year'

print("\nProjected Incidence:")
print(incidence_summary_df)

# Save the detailed results to a multi-sheet Excel file
excel_path_inc = os.path.join(output_dir, "Projected_Incidence.xlsx")
with pd.ExcelWriter(excel_path_inc) as writer:
    male_incident_cases.round(2).to_excel(writer, sheet_name="Male Cases")
    female_incident_cases.round(2).to_excel(writer, sheet_name="Female Cases")
    incidence_summary_df.to_excel(writer, sheet_name="Summary")

print(f"\nSuccessfully saved incidence projection to {excel_path_inc}")



--- Calculating Incidence ---

Projected Incidence:
      Male Incident Cases  Female Incident Cases  Total Incident Cases
Year                                                                  
2023                16640                  13631                 30270
2024                16916                  13869                 30785
2025                17207                  14124                 31331
2026                17490                  14379                 31869
2027                17770                  14636                 32406
2028                18031                  14881                 32912
2029                18274                  15117                 33390
2030                18508                  15343                 33851

Successfully saved incidence projection to output\Projected_Incidence.xlsx


In [19]:

def diabetes_projection_with_io_defaults(base_dir: str = '.',
                                         incidence_filename: str = 'Projected_Incidence.xlsx',
                                         prevalence_filename: str = 'Projected_Prevalence.xlsx',
                                         output_filename: str = 'Projected_Population_Diabetes.xlsx'):
    # Folders relative to project root
    input_dir = os.path.join(base_dir, 'input')
    output_dir = os.path.join(base_dir, 'output')

    # Full paths
    incidence_file = os.path.join(input_dir, incidence_filename)
    prevalence_file = os.path.join(input_dir, prevalence_filename)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_filename)

    # Load inputs
    male_prev = pd.read_excel(prevalence_file, sheet_name='Male Cases', index_col='Age')
    female_prev = pd.read_excel(prevalence_file, sheet_name='Female Cases', index_col='Age')
    male_incid = pd.read_excel(incidence_file, sheet_name='Male Cases', index_col='Age')
    female_incid = pd.read_excel(incidence_file, sheet_name='Female Cases', index_col='Age')

    # Ensure year columns are ints
    male_prev.columns = male_prev.columns.astype(int)
    female_prev.columns = female_prev.columns.astype(int)
    male_incid.columns = male_incid.columns.astype(int)
    female_incid.columns = female_incid.columns.astype(int)

    ages = list(range(0, 100))
    years = list(range(2022, 2031))

    # Initialize outputs
    male_proj = pd.DataFrame(index=ages, columns=years, dtype=float)
    female_proj = pd.DataFrame(index=ages, columns=years, dtype=float)

    # Baseline 2022
    male_proj = male_prev.copy()
    female_proj = female_prev.copy()

    # Project 2023..2030
    for Y in range(2023, 2031):
        # Age 0: only incidence
        male_proj.loc[0, Y] = male_incid.loc[0, Y]
        female_proj.loc[0, Y] = female_incid.loc[0, Y]
        # Ages 1..99: shift cohort + add current incidence
        for a in range(1, 100):
            male_proj.loc[a, Y] = male_proj.loc[a - 1, Y - 1] + male_incid.loc[a, Y]
            female_proj.loc[a, Y] = female_proj.loc[a - 1, Y - 1] + female_incid.loc[a, Y]

    # Summary
    summary_years = list(range(2022, 2031))
    summary = pd.DataFrame(index=summary_years,
                           columns=['Male Prevalent Cases', 'Female Prevalent Cases', 'Total Prevalent Cases'],
                           dtype=float)
    summary.index.name = 'Year'
    summary['Male Prevalent Cases'] = [male_proj[Y].sum() for Y in summary_years]
    summary['Female Prevalent Cases'] = [female_proj[Y].sum() for Y in summary_years]
    summary['Total Prevalent Cases'] = summary['Male Prevalent Cases'] + summary['Female Prevalent Cases']

    # Save to output folder as a single multi-sheet Excel file
    with pd.ExcelWriter(output_path) as writer:
        male_proj.round(2).to_excel(writer, sheet_name='Male Cases')
        female_proj.round(2).to_excel(writer, sheet_name='Female Cases')
        summary.round(2).to_excel(writer, sheet_name='Summary')

    print(f'Input dir: {input_dir}')
    print(f'Output dir: {output_dir}')
    print(f'Saved: {output_path}')

    # Return DataFrames for immediate use
    return male_proj, female_proj, summary


In [21]:
male, female, summary = diabetes_projection_with_io_defaults()
summary


Input dir: .\input
Output dir: .\output
Saved: .\output\Projected_Population_Diabetes.xlsx


,Male Prevalent Cases,Female Prevalent Cases,Total Prevalent Cases
Year,,,
2022,93413.84,69359.75,162773.59
2023,110016.23,82899.12,192915.35
2024,126908.83,96707.72,223616.55
2025,144077.05,110741.19,254818.24
2026,161508.39,124999.90,286508.29
2027,179192.79,139464.26,318657.05
2028,197093.44,154118.90,351212.34
2029,215193.50,168923.32,384116.82
2030,233459.21,183875.16,417334.37
